In [3]:
using JSON
# Specify the file name and location
filename = "small_1.json"
# Read the JSON file
data = JSON.parsefile(filename)

# Extract the details from the JSON
# For example, extract all shops
linear_model = true  # Set to false if you don't want a linear model
shops = data["shops"]
println("Shops:")
println(shops)

# Extract vehicles
vehicles = data["vehicles"]
shops = data["shops"]
parameters = data["parameters"]
constraints = data["constraints"]

for vehicle in vehicles
    if linear_model
        # Set all vehicles to regular if linear_model is true
        println("Vehicle Type: regular")
    else
        # Otherwise, print the actual vehicle type
        println("Vehicle ID: ", vehicle["id"], ", Type: ", vehicle["type"])
    end
end
for constraint in constraints
    println("Constraint: ", constraint)
end
for parameter in parameters
    println("Parameter: ", parameter)
end
for shop in shops
    println("Shop: ", shop)
end



Shops:
Any[Dict{String, Any}("name" => "body", "resequencing_lag" => 3), Dict{String, Any}("name" => "paint", "resequencing_lag" => 3), Dict{String, Any}("name" => "assembly", "resequencing_lag" => 0)]
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicl

In [4]:
resequencing_cost = parameters["resequencing_cost"]
number_of_shops = length(shops)    
for shop in shops
    println("Shop: ", shop)
end
println("Number of shops: ", number_of_shops)
number_of_vehicles = length(vehicles)
println("number_of_vehicles, ",number_of_vehicles)

Shop: Dict{String, Any}("name" => "body", "resequencing_lag" => 3)
Shop: Dict{String, Any}("name" => "paint", "resequencing_lag" => 3)
Shop: Dict{String, Any}("name" => "assembly", "resequencing_lag" => 0)
Number of shops: 3
number_of_vehicles, 54


In [ ]:

# Initialize an empty dictionary to store the mapping
shop_map = Dict{String, Vector{Any}}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
lag_map = Dict{String, Integer}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
# Populate the shop_map with shop names as keys and empty arrays as values
for shop in shops
    shop_map[shop["name"]] = []
    lag_map[shop["name"]] = shop["resequencing_lag"]
    print(lag_map[shop["name"]])
end
n = number_of_vehicles
println("n: ", n)
# Initialize counters or variables if needed

# Assume `model` is defined somewhere before this loop (like in JuMP)
# Example: model = Model(Ipopt.Optimizer)

# Iterate through constraints and associate them with the corresponding shop in shop_map

for constraint in constraints
    name = constraint["shop"]
    if haskey(shop_map, name)
        push!(shop_map[name], constraint)
    else
        println("Warning: Shop name $name in constraints does not exist in shops.")
    end
end

330n: 54


In [6]:
function permutation(x,n)
    permutation = [0 for i in 1:n]  # Initialize the permutation array
    
    for i in 1:n
        for j in 1:n
            if x[i, j] == 1
                permutation[i] = j
            end
        end
    end
    return permutation
end


permutation (generic function with 1 method)

In [7]:
using JuMP,GLPK
model = Model(GLPK.Optimizer)
total_cost = 0
n = number_of_vehicles
for (shop, constraints) in shop_map
    x = @variable(model, [1:n, 1:n], Bin)
    for v in 1:n
        @constraint(model, sum(x[v, t] for t in 1:n) == 1)
        @constraint(model, sum( x[t, v] for t in 1:n) == 1)
    end
    for constraint in constraints
        println("    Shop: ", constraint["shop"])
        if constraint["type"] == "lot_change"
            println("Type: lot_change")
            cl = constraint["cost"]
            k = length(constraint["partition"])
            Ul = constraint["partition"]
            println("UL is ", Ul)
            u = @variable(model, [1:n, 1:k], Bin)
            f = @variable(model, [1:n], Bin)
            
            for t in 1:n
                for o in 1:k
                    sum = 0
                    list = Ul[o]
                    for v in list
                        sum += x[v, t]
                    end
                    @constraint(model, u[t, o] == sum)
                end
            end
            
            for t in 1:n-1
                for o in 1:k
                    @constraint(model, f[t] >= u[t, o] - u[t+1, o])
                end
            end
            total_cost += cl * sum(f)
        end
        if constraint["type"] == "rolling_window"
            println("Type: rolling_window")
            q = @variable(model, [1:n], Bin)
            z = @variable(model, [1:n], Int)
            @constraint(model, z .>= 0)
            Vr = constraint["vehicles"]
            Mr = constraint["max_vehicles"]
            wr = constraint["window_size"]
            cr = constraint["cost"]

            for t in 1:n
                sum = 0
                for v in Vr
                    @constraint(model, q[t] >= x[v, t])
                    sum += x[v, t]
                end
                @constraint(model, q[t] <= sum)
            end

            for t in 1:n
                sum = 0
                for t_prime in t:min(t+wr-1, n)
                    sum += q[t_prime]
                end
                @constraint(model, z[t] >= sum - Mr)
            end
            for t in 1:max(n-wr+1,1) 
                total_cost += cr * z[t]
            end
        end
        if constraint["type"] == "batch_size"
            println("    Type: batch_size")
            mb = constraint["min_vehicles"]
            Mb = constraint["max_vehicles"]
            Vb = constraint["vehicles"]
            cb = constraint["cost"]

            function gamma(t)
                return cb * max(0, max(mb-t, t-Mb))^2
            end
            p = @variable(model, [1:n], Bin)
            m = @variable(model, [1:n, 1:n], Bin)
            l = @variable(model, [1:n, 1:n], Bin)
            j = @variable(model, [1:n, 1:n], Bin)
            g = @variable(model, [1:n, 1:n], Bin)
            for t in 1:n
                sum = 0
                for v in Vb
                    @constraint(model, p[t] >= x[v, t])
                    sum += x[v, t]
                end
                @constraint(model, p[t] <= sum)
            end

            for t in 1:n
                for t_prime in t:n
                    if t >= 2
                        @constraint(model, j[t, t_prime] >= 1 - p[t])
                    end
                    for k in t_prime:t
                        @constraint(model, m[t, t_prime] >= p[t])
                    end
                    if t_prime <= n-1
                        @constraint(model, l[t, t_prime] >= 1 - p[t_prime])
                    end
                end
            end
            for t in 1:n
                for t_prime in t:n
                    @constraint(model, g[t, t_prime] >= m[t, t_prime] + l[t, t_prime] + j[t, t_prime] - 2)
                end
            end
            for t in 1:n-1 
                for t_prime in t:n
                    total_cost += gamma(t_prime-t+1) * g[t, t_prime]
                end
            end
        end 
    end
    @objective(model, Min, total_cost)
    # Optimize model
    optimize!(model)
    # Output results
    if termination_status(model) == MOI.OPTIMAL
        println(termination_status(model))
        x_values = value.(x)
        println("Total cost for shop $shop: $(objective_value(model))")
        println("Optimized values of x:")
        println(x_values)
        println("Permutation: ", permutation(x_values, n))
    else
        println("Solver termination status: ", termination_status(model))
    end
    println("next shop")
end


    Shop: body
    Type: batch_size
    Shop: body
Type: rolling_window
    Shop: body
Type: rolling_window
    Shop: body
    Type: batch_size
    Shop: body
Type: rolling_window


InterruptException: InterruptException:

$
\text{The algorithm converges for the small instances but still do not know if it will for the big instances}. \newline
 \text {The solution also given by this approach is far from being a good solution, we need to discuss what to do after this to solve it.}
$

In [ ]:
using JuMP, GLPK

model = Model(GLPK.Optimizer)

# Initialize parameters and variables
cs = parameters["resequencing_cost"]
total_cost = 0
c = 0
S = number_of_shops
n = number_of_vehicles
tx_t = [[0 for i in 1:n] for i in 1:S]  # Corrected tx_t dimensions to [n, S]
println(S)
lags = [0 for s in 1:S]
x = @variable(model, [1:S, 1:n, 1:n], Bin)
y = @variable(model, [1:S, 1:n], Int)
s = 0

# Loop over shops and constraints
for (shop, constraints) in shop_map
    s += 1
    println("Shop: $shop")
    # Constraints for x variables
    for v in 1:n
        @constraint(model, sum(x[s, v, t] for t in 1:n) == 1)
        @constraint(model, sum(x[s, t, v] for t in 1:n) == 1)
    end

    # Loop over shop constraints
    for constraint in constraints
        println("    Shop: ", constraint["shop"])
        
        if constraint["type"] == "lot_change"
            println("Type: lot_change")
            cl = constraint["cost"]
            k = length(constraint["partition"])
            println("k is ", k)
            Ul = constraint["partition"]
            println("UL is ", Ul)
            
            # Create u and f variables for lot change constraints
            u = @variable(model, [1:n, 1:k], Bin)
            f = @variable(model, [1:n], Bin)
            
            # Constraint on u[t, o]
            for t in 1:n
                for o in 1:k
                    sum = 0
                    list = Ul[o]
                    for v in list
                        sum += x[s, v, t]
                    end
                    @constraint(model, u[t, o] == sum)
                end
            end
            
            # f[t] constraints
            for t in 1:n-1
                for o in 1:k
                    @constraint(model, f[t] >= u[t, o] - u[t+1, o])
                end
            end

            # Add the cost contribution from f[t]
            for t in 1:n 
                total_cost += cl * f[t]
            end
        end
    end
end
# Constraints involving tx_t
for s in 1:S-1
    for v in 1:n
        @constraint(model, y[s,v] >= tx_t[s][v] - tx_t[s+1][v] - lags[s])
    end
end
s = 0
for (shop, constraints) in shop_map
    s += 1
    println("Shop: $shop")
    lag = lag_map[shop]
    lags[s] = lag
    # Calculate tx_t
    for v in 1:n
        sum = 0
        for t in 1:n
            sum += t * x[s, v, t]
        end
    end
end

# Define binary variable b
b = @variable(model, [1:S, 1:n], Bin)
@constraint(model, [i in 1:S, j in 1:n], 1000 * b[i, j] >= y[i, j])

# Add resequencing cost to total_cost
for s in 1:S
    for v in 1:n
        total_cost += cs * b[s, v]
    end
end

# Set the objective
@objective(model, Min, total_cost)

# Optimize the model
optimize!(model)

# Output results
if termination_status(model) == MOI.OPTIMAL
    println(termination_status(model))
    x_values = value.(x)
    println("Optimized values of x:")
    println(x_values)
    # Iterate over all indices of the variable x and print the value of each component
    # Create a vector to hold the regrouped matrices
    regrouped_x = []

    # Iterate over S, and for each S, create a matrix of size n x n
    for i in 1:S
        # Initialize an n x n matrix to store the values for this i
        matrix = zeros(Int, n, n)  # or any other appropriate value type
        
        # Fill the matrix with the corresponding values of x[i, j, k]
        for j in 1:n
            for k in 1:n
                matrix[j, k] = value(x[i, j, k])  # Store the value of x[i, j, k]
            end
        end
        
        # Add the matrix to the regrouped_x array
        push!(regrouped_x, matrix)
    end

    # Print the regrouped matrices (optional)
    for i in 1:S
        println("Matrix for S = $i:")
        println(regrouped_x[i])
        permut = permutation(regrouped_x[i], n)
        println("Permutation: ", permut)
    end
    println("Total cost: $(objective_value(model))")


else
    println("Solver termination status: ", termination_status(model))
end


In [ ]:
using JuMP, GLPK
L = []
model = Model(GLPK.Optimizer)
# Initialize parameters and variables
cs = parameters["resequencing_cost"]
total_cost = 0
c = 0
S = number_of_shops
n = number_of_vehicles
tx_t = [[0 for i in 1:n] for i in 1:S]  # Corrected tx_t dimensions to [n, S]
println(S)
lags = [0 for s in 1:S]
x = @variable(model, [1:S, 1:n, 1:n], Bin)
y = @variable(model, [1:S, 1:n], Int)
s = 0
# Loop over shops and constraints
for (shop, constraints) in shop_map
    s += 1
    println("Shop: $shop")
    # Constraints for x variables
    for v in 1:n
        @constraint(model, sum(x[s, v, t] for t in 1:n) == 1)
        @constraint(model, sum(x[s, t, v] for t in 1:n) == 1)
    end
    # Loop over shop constraints
    for constraint in constraints
        if constraint["type"] == "lot_change"
            pass
            cl = constraint["cost"]
            k = length(constraint["partition"])
            Ul = constraint["partition"]
            # Create u and f variables for lot change constraints
            u = @variable(model, [1:n, 1:k], Bin)
            f = @variable(model, [1:n], Bin)
            # Constraint on u[t, o]
            for t in 1:n
                for o in 1:k
                    sum = 0
                    list = Ul[o]
                    for v in list
                        sum += x[s, v, t]
                    end
                    @constraint(model, u[t, o] == sum)
                end
            end
            for t in 1:n-1
                for o in 1:k
                    @constraint(model, f[t] >= u[t, o] - u[t+1, o])
                end
            end

            # Add the cost contribution from f[t]
            for t in 1:n 
                total_cost += cl * f[t]
            end
        end
        if constraint["type"] == "rolling_window"
            q = @variable(model, [1:n], Bin)
            z = @variable(model, [1:n], Int)
            @constraint(model, z .>= 0)
            Vr = constraint["vehicles"]
            Mr = constraint["max_vehicles"]
            wr = constraint["window_size"]
            cr = constraint["cost"]

            for t in 1:n
                sum = 0
                for v in Vr
                    @constraint(model, q[t] >= x[s,v, t])
                    sum += x[s,v, t]
                end
                @constraint(model, q[t] <= sum)
            end

            for t in 1:n
                sum = 0
                for t_prime in t:min(t+wr-1, n)
                    sum += q[t_prime]
                end
                @constraint(model, z[t] >= sum - Mr)
            end
            for t in 1:max(n-wr+1,1) 
                total_cost += cr * z[t]
            end
        end
        if constraint["type"] == "batch_size"
            mb = constraint["min_vehicles"]
            Mb = constraint["max_vehicles"]
            Vb = constraint["vehicles"]
            cb = constraint["cost"]

            function gamma(t)
                return cb * max(0, max(mb-t, t-Mb))^2
            end
            p = @variable(model, [1:n], Bin)
            m = @variable(model, [1:n, 1:n], Bin)
            l = @variable(model, [1:n, 1:n], Bin)
            j = @variable(model, [1:n, 1:n], Bin)
            g = @variable(model, [1:n, 1:n], Bin)
            for t in 1:n
                sum = 0
                for v in Vb
                    @constraint(model, p[t] >= x[s,v, t])
                    sum += x[s,v, t]
                end
                @constraint(model, p[t] <= sum)
            end

            for t in 1:n
                for t_prime in t:n
                    if t >= 2
                        @constraint(model, j[t, t_prime] >= 1 - p[t])
                    end
                    for k in t_prime:t
                        @constraint(model, m[t, t_prime] >= p[t])
                    end
                    if t_prime <= n-1
                        @constraint(model, l[t, t_prime] >= 1 - p[t_prime])
                    end
                end
            end
            for t in 1:n
                for t_prime in t:n
                    @constraint(model, g[t, t_prime] >= m[t, t_prime] + l[t, t_prime] + j[t, t_prime] - 2)
                end
            end
            for t in 1:n-1 
                for t_prime in t:n
                    total_cost += gamma(t_prime-t+1) * g[t, t_prime]
                end
            end
        end 
        
        
    end
end
# Constraints involving tx_t
for s in 1:S-1
    for v in 1:n
        @constraint(model, y[s,v] >= tx_t[s][v] - tx_t[s+1][v] - lags[s])
    end
end
s = 0
for (shop, constraints) in shop_map
    s += 1
    lag = lag_map[shop]
    lags[s] = lag
    # Calculate tx_t
    for v in 1:n
        sum = 0
        for t in 1:n
            sum += t * x[s, v, t]
        end
    end
end
# Define binary variable b
b = @variable(model, [1:S, 1:n], Bin)
@constraint(model, [i in 1:S, j in 1:n], 1000 * b[i, j] >= y[i, j])

# Add resequencing cost to total_cost
for s in 1:S
    for v in 1:n
        total_cost += cs * b[s, v]
    end
end
# Set the objective
@objective(model, Min, total_cost)
# Optimize the model
optimize!(model)
# Output results
if termination_status(model) == MOI.OPTIMAL
    println(termination_status(model))
    x_values = value.(x)
    println(x_values)
    # Iterate over all indices of the variable x and print the value of each component
    # Create a vector to hold the regrouped matrices
    regrouped_x = []
    # Iterate over S, and for each S, create a matrix of size n x n
    for i in 1:S
        # Initialize an n x n matrix to store the values for this i
        matrix = zeros(Int, n, n)  # or any other appropriate value type
        
        # Fill the matrix with the corresponding values of x[i, j, k]
        for j in 1:n
            for k in 1:n
                matrix[j, k] = value(x[i, j, k])  # Store the value of x[i, j, k]
            end
        end
        # Add the matrix to the regrouped_x array
        push!(regrouped_x, matrix)
    end
    # Print the regrouped matrices (optional)
    for i in 1:S
        println("Matrix for S = $i:")
        println(regrouped_x[i])
        permut = permutation(regrouped_x[i], n)
        push!(L, permut)
        println("Permutation: ", permut)
    end
    println("Total cost: $(objective_value(model))")
else
    println("Solver termination status: ", termination_status(model))
end


In [ ]:
function swap_neighbor(perm::Vector{Int})
    # Ensure the permutation has at least two elements
    if length(perm) < 2
        throw(ArgumentError("The permutation must have at least two elements to perform a swap"))
    end
    
    # Randomly choose two distinct positions (indices) in the permutation
    idx1, idx2 = rand(1:length(perm)), rand(1:length(perm))
    
    # Ensure the indices are different
    while idx1 == idx2
        idx2 = rand(1:length(perm))
    end
    
    # Swap the elements at idx1 and idx2
    perm_copy = copy(perm)  # To preserve the original permutation
    perm_copy[idx1], perm_copy[idx2] = perm_copy[idx2], perm_copy[idx1]
    
    return perm_copy
end


In [ ]:
function swap_neighbor(perm::Vector{Int})
    # Ensure the permutation has at least two elements
    if length(perm) < 2
        throw(ArgumentError("The permutation must have at least two elements to perform a swap"))
    end
    
    # Randomly choose two distinct positions (indices) in the permutation
    idx1, idx2 = rand(1:length(perm)), rand(1:length(perm))
    
    # Ensure the indices are different
    while idx1 == idx2
        idx2 = rand(1:length(perm))
    end
    
    # Swap the elements at idx1 and idx2
    perm_copy = copy(perm)  # To preserve the original permutation
    perm_copy[idx1], perm_copy[idx2] = perm_copy[idx2], perm_copy[idx1]
    
    return perm_copy
end
function gamma(t)
    return cb * max(0, max(mb-t, t-Mb))^2
end


In [ ]:
function calculate_cost(perm::Vector{Int}, Cl::Array{Array{Int}}, Ul::Array{Array{Vector{Int}}}, 
                       Vr::Array{Array{Int}}, Mr::Array{Int}, wr::Array{Int}, 
                       cr::Array{Int}, mb::Array{Int}, Mb::Array{Int}, 
                       cb::Array{Int}, Vb::Array{Array{Int}})

    total_cost = 0

    # Calculate the cost for lot change constraints (Cl and Ul)
    if length(Cl) > 0 && length(Ul) > 0
        for (i, l) in enumerate(Cl)
            for s in l
                for t in 1:length(perm)-1
                    # Check if consecutive vehicles belong to different lots
                    if Ul[i][perm[t]] != Ul[i][perm[t+1]]
                        total_cost += Ul[i][s]  # Add penalty associated with the lot change
                    end
                end
            end
        end
    end

    # Calculate the cost for rolling window constraints (Vr, Mr, wr, cr)
    if length(Vr) > 0 && length(Mr) > 0 && length(wr) > 0 && length(cr) > 0
        for (r_idx, r) in enumerate(Vr)
            max_violations = 0
            for t in 1:(length(perm) - wr[r_idx] + 1)
                window = perm[t:t+wr[r_idx]-1]
                violation_count = sum(v in r for v in window)
                if violation_count > Mr[r_idx]
                    max_violations += (violation_count - Mr[r_idx])^2 * cr[r_idx]
                end
            end
            total_cost += max_violations
        end
    end

    # Calculate the cost for batch size constraints (Vb, mb, Mb, cb)
    if length(Vb) > 0 && length(mb) > 0 && length(Mb) > 0 && length(cb) > 0
        for (b_idx, b) in enumerate(Vb)
            max_violations = 0
            for t in 1:(length(perm) - 1)
                batch = perm[t:t+1]
                batch_size = length(batch)
                if batch_size < mb[b_idx] || batch_size > Mb[b_idx]
                    max_violations += (max(0, mb[b_idx] - batch_size, batch_size - Mb[b_idx]))^2 * cb[b_idx]
                end
            end
            total_cost += max_violations
        end
    end

    return total_cost
end


In [ ]:
for (shop, constraints) in shop_map
    print("shop :",shop)
    Cl = []
    Ul = []
    Vr =[]
    Mr = []
    wr = []
    cr = []
    mb = []
    Mb = []
    cb = []
    Vb = []
    for constraint in constraints
        if constraint["type"] == "lot_change"
            cl = constraint["cost"]
            Ul_0= constraint["partition"]
            push!(Ul,Ul_0)
            push!(Cl,cl)
        end
        if constraint["type"] == "rolling_window"

            Vr_0 = constraint["vehicles"]
            Mr_0 = constraint["max_vehicles"]
            wr_0 = constraint["window_size"]
            cr_0 = constraint["cost"]
            push!(Vr,Vr_0)
            push!(Mr,Mr_0)
            push!(wr,wr_0)
            push!(cr,cr_0)
        end
        if constraint["type"] == "batch_size"
            mb_0 = constraint["min_vehicles"]
            Mb_0 = constraint["max_vehicles"]
            Vb_0 = constraint["vehicles"]
            cb_0 = constraint["cost"]
            push!(mb,mb_0)
            push!(Mb,Mb_0)
            push!(Vb,Vb_0)
            push!(cb,cb_0)
        end 
        
        
    end
end
